In [1]:
# ============================================================
# Notebook setup
# ============================================================

%load_ext autoreload
%autoreload 2

# Control figure size
interactive_figures = True
if interactive_figures:
    # Normal behavior
    %matplotlib widget
    figsize=(9, 3)
else:
    # PDF export behavior
    figsize=(14, 5)

from util import er
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from ortools.sat.python import cp_model
import copy

# Kidney Exchange Problem

## Kidney Exchange

**Let's consider now a problem from a different (medical domain)**

We consider organ transplantation from living donors (e.g. kidney)

* Incompatibility issues are major bottleneck
* ...But sometimes we are in this kind of situation:

<center><img src="assets/paired-donation.jpg" width="500px"/></center>

* There are two willing donor, with incompatible recipients
* ...But an exchange can be performed!

## Kidney Exchange

**Operationally, it works as follows:**

* Recipient-donor pairs enter a _kidney exchange program_
* Periodically, the pairs must be matched so as to enable transplantation
  - Matches can be simple (2 pairs) or more complex (3 or more pairs)
* Surgery is performed within a short time time frame
  - Usually this sets a limit on the size of each chain

**The matching problem is know as _Kidney Exchange Problem (KEP)_**

* Consists in assigning donor to recipients so as to form closed chains
* ...And so as to maximize (typically) the number of transplants
* It is an _online problem_, but we will tackle in its offline version

## Problem Formulation

**The KEP admits a graph-based formulation**

* Recipient-donor _pairs_ $(r_i, d_i)$ in the programs can be seen as _nodes in a graph_
* The graph contains an arc from pair $i$ to pair $j$ iff the $d_i$ is compatible with $r_j$

<center><img src="assets/cgraph.png" width="250px"/></center> 

* In the example there are four pairs
* The donor in pair 1 is compatible with the recipient in pair 2, and so on

## Problem Formulation

**From this perspective, the KEP consists in _selecting cycles_ in the graph**

* Only cycles up to a maximum length are considered
* The weight of a cycle is given by its number of nodes/arcs
* The objective is to maximize the weight of the selected cycles
* Cycles are mutually exclusive if they share at least one node

**The key ideas are clear, but translating them to a model is another story**

* Defining a combinatorial model for _building cycles_ can be complicated
  - E.g. Traveling Salesaman Problem, all Vehicle Routing Problem variants...
* But what if we had access to a _precomputed set of cycles_?

**This is the key idea in the so-called _cycle formulation_**

## Cycle Formulation

**The _cycle formulation_ consists in the following Integer Program**

$$\begin{align}
\max z & = \sum_{i=j}^n w_j x_j \\
\text{s.t. } & \sum_{j=1}^n a_{ij} x_j \leq 1 & \forall i = 1..m \\
& x_j \in \{0, 1\} & \forall j = 1..n
\end{align}$$

* $m$ is the number of pairs, $n$ of cycles
* $w_j$ is the weight of cycle $j$ (i.e. its number of nodes)
* $a_{ij} = 1$ iff node $i$ belongs to cycle $j$ (and $a_{ij} = 0$ otherwise)
* The maximum length constraint is handle when generating the set of cycles

## Generating the Benchmark

**We will try to build a cycle formulation approach**

...But first we need to obtain a benchmark (a dataset)

* We will use synthetic data, obtain via the following function:

In [2]:
pairs, arcs, aplus = er.generate_compatibility_graph(size=12, seed=2)

* The function generates a fixed number of pairs
* ...And their compatibility graphs

The approach is designed to be reasonably realistic

## Generating the Benchmark 

**The generated pairs are associated to incompatible _blood types_**

In [4]:
pairs

{0: pair(recipient='B+', donor='A+'),
 1: pair(recipient='B+', donor='A+'),
 2: pair(recipient='O+', donor='B+'),
 3: pair(recipient='A+', donor='B+'),
 4: pair(recipient='O+', donor='A+'),
 5: pair(recipient='O+', donor='A-'),
 6: pair(recipient='A-', donor='O+'),
 7: pair(recipient='A+', donor='B+'),
 8: pair(recipient='B+', donor='A+'),
 9: pair(recipient='O+', donor='A+'),
 10: pair(recipient='O+', donor='A+'),
 11: pair(recipient='A-', donor='A+')}

* The blood type prevalence reflects the Italian distribution
* Incompatibility is mainly due to blood type
* ...Plus a number of more complex, but less impacting, factors

## Generating the Benchmark

**Arcs are first determined based on blood type compatibility**

* Then a small (random) fraction of them (5%) is removed
* ...So as to simulate the other compatibility factors

In [5]:
aplus

{0: [3, 7],
 1: [3, 7],
 2: [0, 1, 8],
 3: [0, 1, 8],
 4: [3, 7],
 5: [3, 6, 7, 11],
 6: [0, 1, 2, 3, 4, 5, 7, 8, 9, 10],
 7: [0, 1, 8],
 8: [3, 7],
 9: [3, 7],
 10: [3, 7],
 11: [3, 7]}

## Enumerating Cycles

**We enumerate cycles using simple Depth First Search with limited depth**

```python
def cycle_next(seq, nsteps, aplus, cycles, cap=None):
    node = seq[-1]
    successors = np.array(aplus[node]) # Consider all possible successors
    np.random.shuffle(successors) # ...in randomized order
    for dst in successors:
        # Early exit if the capacity has been exceeded
        if cap is not None and len(cycles) >= cap: return
        if dst == seq[0] and dst == min(seq): # close the cycle
            cycles.add(tuple(seq))
        elif nsteps > 0 and dst not in seq:
            cycle_next(seq+[dst], nsteps-1, aplus, cycles, cap) # recursive call
```

* Cycles are stored as tuples, which mean that the node ordering matters
* ...So we take only the ordering that starts with the minimum index
* There is a capacity parameter to limit the number of enumerated cycles

## Enumerating Cycles

**We use a second function to start the enumeration from all possible sources**

```python
def find_all_cycles(aplus, max_length, cap=None, seed=42):
    cycles = set()
    roots = np.array(list(aplus.keys()))
    np.random.seed(seed)
    np.random.shuffle(roots)
    for node in roots:
        if cap is None or len(cycles) < cap:
            cycle_next([node], max_length-1, aplus, cycles, cap)
    return list(cycles)
```

**We can now enumerate the cycles for our graph (HP: max length of 4)**

In [6]:
cycles = er.find_all_cycles(aplus, max_length=4, cap=None)
print(sorted(cycles))

[(0, 3), (0, 3, 1, 7), (0, 3, 8, 7), (0, 7), (0, 7, 1, 3), (0, 7, 8, 3), (1, 3), (1, 3, 8, 7), (1, 7), (1, 7, 8, 3), (3, 8), (5, 6), (7, 8)]


## Cycle Formulation - Implementation

**Once we have all cycles, we can build the Cycle Formulation model**

```python
def cycle_formulation(pairs, cycles, tlim=None, verbose=1):
    infinity, ncycles, npairs = slv.infinity(), len(cycles), len(pairs)
    slv = pywraplp.Solver.CreateSolver('CBC') # Build the solver
    cpp = {i:[] for i in range(npairs)} # group cycles by pair
    for j, cycle in enumerate(cycles):
        for i in cycle: cpp[i].append(j)
    x = [slv.IntVar(0, 1, f'x_{j}') for j in range(ncycles)] # variables
    for i in range(npairs): # constraints
        slv.Add(sum(x[j] for j in cpp[i]) <= 1)
    slv.Maximize(sum(len(c) * x[j] for j, c in enumerate(cycles))) # objective
    if tlim is not None: # time limit
        slv.SetTimeLimit(1000*tlim)
    status = slv.Solve() # solve
    # Extract results and return
    ...
```

## Cycle Formulation - Implementation

**We use [the CBC solver](https://github.com/coin-or/Cbc)**

```python
def cycle_formulation(pairs, cycles, tlim=None, verbose=1):
    infinity, ncycles, npairs = slv.infinity(), len(cycles), len(pairs)
    slv = pywraplp.Solver.CreateSolver('CBC') # Build the solver
    ...
```

* It's the fastest MIP solver with a fully permissive license

**Variables are build with `IntVar`, constraints posted with `Add`**

```python
def cycle_formulation(pairs, cycles, tlim=None, verbose=1):
    ...
    x = [slv.IntVar(0, 1, f'x_{j}') for j in range(ncycles)] # variables
    for i in range(npairs): # constraints
        slv.Add(sum(x[j] for j in cpp[i]) <= 1)
    ...
```

* The `cpp` dictionary contains cycles, grouped by the pair/node they use

## Cycle Formulation - Implementation

**We set the objective with `Maximize` or `Minimize`**

```python
def cycle_formulation(pairs, cycles, tlim=None, verbose=1):
    ...
    slv.Maximize(sum(len(c) * x[j] for j, c in enumerate(cycles))) # objective
    if tlim is not None: # time limit
        slv.SetTimeLimit(1000*tlim)
    ...
```

* Time limits are enforced with `SetTimeLimit`

**We can now solve the cycle formulation:**

In [7]:
pairs, arcs, aplus = er.generate_compatibility_graph(size=12, seed=2)
cycles = er.find_all_cycles(aplus, max_length=4, cap=None)
sol, tme, _ = er.cycle_formulation(pairs, cycles, tlim=10, verbose=1)
print({k for k, v in sol.items() if v != 0 and k != 'objective'})

Solution time: 0.013, objective value: 6.0 (optimal)
{'x_2', 'x_1', 'x_6'}


## Scalability Issue

**The main drawback of the cycle formulation is the limited scalability**

* The number of cycles grows with the graph size
* ...with a high-degree polynomial law, i.e. $O(n^{\text{max length}})$
* The enumeration becomes _more expensive_ and the model becomes _larger_

**Both can quickly become major bottlenecks**

In [8]:
pairs2, arcs2, aplus2 = er.generate_compatibility_graph(size=150, seed=2)
print('>>> Size 150, enumeration time')
%time cycles2 = er.find_all_cycles(aplus2, max_length=4, cap=None)
print(f'Number of cycles: {len(cycles2)}')
print('>>> Size 150, solution time')
%time _, _, _ = er.cycle_formulation(pairs2, cycles2, tlim=10, verbose=0)

>>> Size 150, enumeration time
CPU times: user 10.4 s, sys: 0 ns, total: 10.4 s
Wall time: 10.4 s
Number of cycles: 43206
>>> Size 150, solution time
CPU times: user 3.43 s, sys: 23.2 ms, total: 3.45 s
Wall time: 3.46 s


Apparently, this will completely kill our scalability: is there a way around?

# Column Generation

## Column Generation and Dual Multipliers

**_Column Generation_ is a technique for solving problems with many variables**

* Main idea: dynamically generate only the variables that are _needed_
* I.e. that have a chance of being part of an optimal solution

The technique is mainly designed for Linear Programs

**How is this achieved?**

Say we have a Linear Program  in the form:
$$\min \{cx \mid Ax \geq b, x\geq 0\}$$

* This can be solved in polynomial time with Interior Point algorithm
* ...Or in pseudo-polynomial time with the Simplex algorithm
* Both provide optimal values $x^*$ for the _primal variables_
* ...But also a vector of optimal _dual multipliers $\lambda$_ (or _dual variables_)

## Column Generation and Dual Multipliers

**The multipliers stem from an alternative problem formulation**

...Where we minimize a cost that includes _penalties_ for violating the constraints:

$$
cx + \lambda^T (b - Ax)
$$

* $\lambda$ is a vector of multipliers (a.k.a. dual variables)
* ...Each one is associated to a problem constraint

**Remember we are supposed to have $Ax \geq b$. Therefore:**

* If $b_i - A_ix \geq 0$ it means we have a _constraint violation_
  - ...And we receive a _penalty_ given by $\lambda_i \times$ the violation
* If $b_i - A_ix < 0$ the constraint is _satisfied with a slack_
  - ...And we receive a _reward_ given by $\lambda_i \times$ the violation




## Column Generation and Dual Multipliers

**When an LP solves problem, it also solves its _dual version_**

* The goal is to set the dual multipliers _just right_
* ...So that they discourage violating the constraints
* ...And so that we accumulate no unnecessary reward

**Without getting into the details...**

...For any optimal solution $x^*$ we have that:

* If a constraint $i$ is _satisfied with a slack_, then $\lambda_i = 0$
  - In the alternative formulation, we receive no reward for the slack
  - ...Since we need to incentive for satisfying the constraint
* If a constraint $i$ is _tight_, then $\lambda_i \geq 0$:
  - Any violation would incur a penalty proportional to $\lambda_i$
  - ...So there is an incentive for not violating the constraints

## Cycle Formulation LP

**Let's see this in action on our problem**

* First, we need to consider the LP relaxation of our integer program
* ...And we need to rewrite it in standard form:

$$\begin{align}
\min z & = \sum_{i=j}^n -w_j x_j \\
\text{s.t. } & \sum_{j=1}^n -a_{ij} x_j \geq -1 & \forall i = 1..m \\
& x_j \geq 0 & \forall j = 1..n
\end{align}$$

Besides removing the integrality constraints, we:

* ...Switched the optimization direction (from max to min)
* ...Switched the direction of the constraints

## Cycle Formulation LP

**We can now modify our model-building function**

The function should build a Linear Program in the expected format:

```python
def cycle_formulation(pairs, cycles, tlim=None, relaxation=False, verbose=1):
    if relaxation: # Build the solver
        slv = pywraplp.Solver.CreateSolver('CLP')
    else:
        slv = pywraplp.Solver.CreateSolver('CBC')
    ...
```

* We have added an optional `relaxation` parameter
* We use [the CLP solver](https://github.com/coin-or/Clp) (the fastest with a fully permissive license)

## Cycle Formulation LP

**We need to build continuous, rather than integer, variables**

```python
def cycle_formulation(pairs, cycles, tlim=None, relaxation=False, verbose=1):
    ...
    if relaxation: # variables
        x = [slv.NumVar(0, infinity, f'x_{j}') for j in range(ncycles)]
    else:
        x = [slv.IntVar(0, 1, f'x_{j}') for j in range(ncycles)]
```

**We also post the constraints in the format used in our theoretical arguments:**

```python
def cycle_formulation(pairs, cycles, tlim=None, relaxation=False, verbose=1):
    ...
    for i in range(npairs): # constraints
        if relaxation:
            slv.Add(-sum(x[j] for j in cpp[i]) >= -1)
        else:
            slv.Add(sum(x[j] for j in cpp[i]) <= 1)
```

## Cycle Formulation LP

**The same goes for the problem objective**

```python
def cycle_formulation(pairs, cycles, tlim=None, relaxation=False, verbose=1):
    obj = sum(len(c) * x[j] for j, c in enumerate(cycles))
    if relaxation: # objective
        slv.Minimize(-obj)
    else:
        slv.Maximize(obj)
```

**Once we have a solution, we can obtain the dual multipliers from the solver**

```python
def cycle_formulation(pairs, cycles, tlim=None, relaxation=False, verbose=1):
    ...
    duals = None
    if status in (slv.OPTIMAL, slv.FEASIBLE):
        ...
        if relaxation:
            duals = np.array([c.dual_value() for c in slv.constraints()])
    ...
```

## Cycle Formulation LP

**Now, let's try to solve the LP**

In [8]:
pairs, arcs, aplus = er.generate_compatibility_graph(size=12, seed=2)
cycles = er.find_all_cycles(aplus, max_length=4, cap=None)
sol, tme, duals = er.cycle_formulation(pairs, cycles, tlim=10, verbose=1, relaxation=True)
for i, c in enumerate(cycles):
    if sol[f'x_{i}'] == 1: print(c)
print(f'Dual multipliers: {duals}')

Solution time: 0.001, objective value: -6.0 (optimal)
(7, 8)
(5, 6)
(0, 3)
Dual multipliers: [0. 0. 0. 2. 0. 2. 0. 2. 0. 0. 0. 0.]


* We have one multiplier per constraint, i.e. one per graph node in our case
* The non-zero $\lambda$ are associated to nodes used by the selected cycles
  - ...Meaning that their associated constraints are tight
* The cost is negative, since we have negated the original objective formula

## Reduced Costs

**Now, let $x$ be a feasible LP solution, and $\lambda$ its dual multiplier vector**

The multipliers can be used to compute the "gradient of the constrained problem"

* Technically, we will use the _gradient of the Lagrangian relaxation_
* Such gradient comes as a _closed-form_ formula, given by:

$$
\nabla_x\left(cx + \lambda (b - Ax)\right)
$$

* The expression $b - Ax$ represent the violation/slack for constraints $Ax \geq b$


**The gradient can be rewritten as:**
$$
c - \lambda A
$$

* Each component this gradient is known as _reduced cost_
* If $x^*$ is optimal, the gradient (i.e. the reduced costs) will be null

## Reduced Costs

**Now, for a _suboptimal_ solution $x$, the gradient will be _non null_**

I.e. there will be negative components (reduced costs) in:
$$
c - \lambda A
$$

Actually, this is just a _necessary condition_:

* Moving in the gradient direction may improve the cost or keep it unaltered
  - ...It will never make it worse
* Overall, there is a chance for improving $x$ _only if some reduced cost is negative_

**The reduced costs come in closed-form**

* Given $\lambda$, for computing the reduced costs we just need the the coefficients in $A$
* This mean we can compute the reduced costs
* ...Also for _variables that are not yet in the problem_

## Reduced Costs and Column Generation

**This observation suggest as criterion for dynamically adding variables**

<center><img src="assets/cg.png" width="900px"/></center>

* We start by solving an LP with a _subset of the variables_
  - Its solution will be _feasible_, but _not necessarily optimal_
* We consider the remaining variables (with their coefficients in $A$)
  - ...And we compute their _reduced costs_ (pricing problem)
* If there are no _variables with negative r.c._, we know our LP solution is optimal
* Otherwise, we repeat add such variables and repeat the process


## Pricing Problem

**In our case the pricing problem should consider (in principle) all cycles**

This is an issue, since enumeration can be very expensive

* But we _do not need to enumerate_!
* ...We can focus on _the most negative red. costs_

This is enough to check whether there is any negative reduced cost

**Let's have a look at our specific reduced costs structure:**

$$c - \lambda^* A = \left(\begin{array}{c}
\vdots \\
-w_j - \sum_{i=1}^m -a_{ij} \lambda_i\\
\vdots
\end{array}\right) = \left(\begin{array}{c}
\vdots \\
\sum_{i=1}^m a_{ij} (-1 + \lambda_i)\\
\vdots
\end{array}\right)$$

* The equivalence holds since $w_j = \sum_{i=1}^m a_{ij}$ (num. nodes in the cycle)

## Pricing Problem

**Let's try to understand this a bit better**

$$\left(\begin{array}{c}
\vdots \\
\sum_{i=1}^m a_{ij} (-1 + \lambda^*_i)\\
\vdots
\end{array}\right)$$

* If we include node $i$ in a cycle $j$, then $a_{ij} = 1$
* Therefore, we incur a cost of $-1 + \lambda^*_i$

**So, searching for the most negative reduced costs...**

* ...Is equivalent to searching for _minimum weight cycles_
* ...With weights given by: $-1 + \lambda^*_i$

We also have a constraint on the maximum number of nodes per cycles

## Constrained Minimum Cycle Weight 

**We will base our pricing algorithm on a Time Unfolded version of our graph**

* An unfolded graph contains one copy of each original node per time unit
* In our case, time units correspond to possible cycle lengths 

For our example graph, we get:

<center><img src="assets/tug0.png" width="500px"/></center>

Since the maximum length is 4, there is no need to unfold beyond that

## Constrained Minimum Cycle Weight 

**The time-unfolded graph is _layered_**

* There are no arcs between nodes associated to the same time unit
* Arcs connect node associated to contiguous time units

<center><img src="assets/tug1.png" width="500px"/></center>

Since the graph is acyclic, shortest paths can be found using _Dijkstra's algorithm_

## Constrained Minimum Cycle Weight 

**We can process _one layer at a time_**

* We start at layer 1, from a given _root node_ (1, in the figure)
* We consider all outgoing arcs

<center><img src="assets/tug2.png" width="500px"/></center>

We _update the shortest path_ to the destination nodes as usual in Dijkstra's 

## Constrained Minimum Cycle Weight 

**We then start from all visited nodes, and proceed as before**

* If we end up _visiting the root node again_, we have found a cycle
* This is a shortest cycle including the root node, for the current length

<center><img src="assets/tug3.png" width="500px"/></center>

We _store_ all these cycles (in this case, we store the cycle 1-4 for the path 1-4-1)

## Constrained Minimum Cycle Weight 

**Nodes that close a cycle count as non-visited**

* If we end up visiting a non-root node that is on the shortest path
* ...Then we have found a path with a sub-cycles

<center><img src="assets/tug4.png" width="500px"/></center>

We _do not store_ such paths (e.g., we store a cycle for 1-2-3-1, but not 1-2-3-2)

## Constrained Minimum Cycle Weight 

**We proceed until that maximum length is reached**

* ...Or until a given layer contains no visited nodes
* Then we can restart from another root node

<center><img src="assets/tug4.png" width="500px"/></center>

In the next restart, we can _ignore all arcs pointing to already considered roots_

* Since all shortest cycles containing those nodes have already been found

## Constrained Minimum Cycle Weight - Implementation

**The bulk of the computation is done by this function**

```python
def shortest_cycles_from_root(root, aplus, weights, max_len):
    spt = {root: {root}} # initial shortest paths
    dst = {root: weights[root]} # shortest path distances
    cycles, ccosts = [], []
    for k in range(max_len): # loop over the possible cycle lengths
        ndst, nspt = {}, {}
        for i in dst: # process all visited nodes
            for j in aplus[i]: # loop over outgoing arcs
                if j == root: # detect cycles
                    cycles.append(spt[i])
                    ccosts.append(dst[i])
                elif j in spt[i]: # skip subcycles
                    continue 
                elif j not in ndst or dst[i] + weights[j] < ndst[j]: # Dijkstra update
                    ndst[j] = dst[i] + weights[j]
                    nspt[j] = spt[i] | {j}
        dst, spt = ndst, nspt
    return cycles, ccosts
```

## Constrained Minimum Cycle Weight - Implementation

**Initially, the only shortest path include the root alone**

```python
def shortest_cycles_from_root(root, aplus, weights, max_len):
    spt = {root: {root}} # initial shortest paths
    dst = {root: weights[root]} # shortest path distances
```

**We process one layer at a time:**

```python
def shortest_cycles_from_root(root, aplus, weights, max_len):
    ...
    cycles, ccosts = [], []
    for k in range(max_len): # loop over the possible cycle lengths
        ...
    return cycles, ccosts
```

* At the end of the computation we return the shortest paths
* ...And their costs/weights

## Constrained Minimum Cycle Weight - Implementation

**At each layer, we build the shortest paths to the next layer**

```python
def shortest_cycles_from_root(root, aplus, weights, max_len):
    ...
    for k in range(max_len): # loop over the possible cycle lengths
        ndst, nspt = {}, {}
        for i in dst: # process all visited nodes
            for j in aplus[i]: # loop over outgoing arcs
                ...
        dst, spt = ndst, nspt
    ...
```

* We process all "visited" nodes, which are in `dst`
* I.e. those having a valid shortest path for the current length
* Then we loop over all their outgoing arcs

## Constrained Minimum Cycle Weight - Implementation

**How we deal with each outgoing arc depends on the current situation**

```python
def shortest_cycles_from_root(root, aplus, weights, max_len):
            ...
            for j in aplus[i]: # loop over outgoing arcs
                if j == root: # detect cycles
                    cycles.append(spt[i])
                    ccosts.append(dst[i])
                elif j in spt[i]: # skip subcycles
                    continue 
                elif j not in ndst or dst[i] + weights[j] < ndst[j]: # Dijkstra update
                    ndst[j] = dst[i] + weights[j]
                    nspt[j] = spt[i] | {j}
            ...
```

* If we detect a cycle to the root, we store it
* If we detect a sub-cycle, we disregard it
* Otherwise, we keep building the shortest path using Dijkstra's method

## Constrained Minimum Cycle Weight - Implementation

**The previous function is called for each possible root**

```python
def shortest_cycles(aplus, weights, max_len):
    aminus = {i:[] for i in aplus} # graph in backward star format
    for i, alist in aplus.items():
        for j in alist: aminus[j].append(i)
    aplus = copy.deepcopy(aplus) # copy of the forward star
    cycles, ccosts = [], []
    for root in range(len(weights)): # loop over possible roots
        # Collect shortest paths from this root
        tcl, tct = shortest_cycles_from_root(root, aplus, weights, max_len)
        cycles += tcl
        ccosts += tct
        for i in aminus[root]: # remove all forward arcs twd the processed root
            aplus[i].remove(root)
    return cycles, ccosts
```

* After a root is processed, we have all shortest cycles involving that node
* ...Therefore we can disregard it in all future iterations

## Constrained Minimum Cycle Weight

**The process returns (at most) one cycle per root node and per valid weight**

For our example graph, we have:

In [9]:
weights = -np.ones(len(pairs)) + duals
scl, sct = er.shortest_cycles(aplus, weights, max_len=4)
print(scl)
print(sct)

[{0, 3}, {0, 7}, {0, 1, 3, 7}, {1, 3}, {1, 7}, {8, 1, 3, 7}, {8, 3}, {5, 6}, {8, 7}]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


* All shortest cycles have non-negative reduced costs
* This is expected since the dual multiplier refer to an optimal solution

**Focusing on minimum weight cycles gives a _massive speed improvement_:**

In [10]:
pairs2, arcs2, aplus2 = er.generate_compatibility_graph(size=150, seed=2)
%time cycles2, _ = er.shortest_cycles(aplus2, weights=-np.ones(len(pairs2)), max_len=4)

CPU times: user 92.5 ms, sys: 0 ns, total: 92.5 ms
Wall time: 92.3 ms


## Column Generation

**We can now inspect the column generation method itself**

```python
def cycle_formulation_cg(pairs, aplus, max_len, itcap=10, tol=1e-3, verbose=1):
    weights = -np.ones(len(pairs)) # initial cycle pool
    cycles, _ = er.shortest_cycles(aplus, weights, max_len=max_len)
    converged = False # main loop
    for itn in range(itcap):
        sol, stime, duals = er.cycle_formulation(pairs, cycles, verbose=0, relaxation=True)
        if verbose > 0: ...
        weights = -np.ones(len(pairs)) + duals # shortest paths
        scl, sct = er.shortest_cycles(aplus, weights, max_len=max_len)
        nrc_cycles = [scl[i] for i, c in enumerate(sct) if c < -tol] # negative r.c.
        if verbose > 0: ...
        if len(nrc_cycles) == 0: # no improvement possible
            converged = True
            break
        else: cycles += nrc_cycles # add new arcs
    return cycles, converged
```

## Column Generation

**The initial pool of variables corresponds to all shortest cycles**

```python
def cycle_formulation_cg(pairs, aplus, max_len, itcap=10, tol=1e-3, verbose=1):
    weights = -np.ones(len(pairs)) # initial cycle pool
    cycles, _ = er.shortest_cycles(aplus, weights, max_len=max_len)
```

* The cycle weight is just the number of nodes

**Then we start the main loop**

```python
def cycle_formulation_cg(pairs, aplus, max_len, itcap=10, tol=1e-3, verbose=1):
    ...
    converged = False # main loop
    for itn in range(itcap):
        ...
    return cycles, converged
```

* At the end we return the optimized cycle pool, plus convergence flag

## Column Generation

**At each iteration, we solve the LP relaxation**

```python
def cycle_formulation_cg(pairs, aplus, max_len, itcap=10, tol=1e-3, verbose=1):
    ...
    for itn in range(itcap):
        sol, stime, duals = er.cycle_formulation(pairs, cycles, verbose=0, relaxation=True)
        if verbose > 0: ...
```

**Then we find all shortest cycles**

```python
def cycle_formulation_cg(pairs, aplus, max_len, itcap=10, tol=1e-3, verbose=1):
    ...
    for itn in range(itcap):
        ...
        weights = -np.ones(len(pairs)) + duals # shortest paths
        scl, sct = er.shortest_cycles(aplus, weights, max_len=max_len)
        ...
```

## Column Generation

**Then we detect the cycles with negative reduced costs**

```python
def cycle_formulation_cg(pairs, aplus, max_len, itcap=10, tol=1e-3, verbose=1):
    ...
    for itn in range(itcap):
        ...
        nrc_cycles = [scl[i] for i, c in enumerate(sct) if c < -tol] # negative r.c.
        if verbose > 0: ...
        if len(nrc_cycles) == 0: # no improvement possible
            converged = True
            break
        else: cycles += nrc_cycles # add new arcs
```

* LP solvers operate withing tolerances, so it's a good idea to use one
* If there are not cycles with negative r.c. we have converged
* Otherwise, we add all arcs with negative r.c. to the problem

Adding multiple arcs is usually a good idea in Column Generation

## Column Generation - Correctness

**It's time to test the approach. We will initially focus on correctness**

We generate a graph:

In [26]:
pairs, arcs, aplus = er.generate_compatibility_graph(size=100, seed=2)

Then we solve the GC formulation:

In [27]:
cycles_cg, _  = er.cycle_formulation_cg(pairs, aplus, max_len=4, itcap=10)

(CG, it. 0), #cycles: 839, time: 0.038, relaxation objective: -36.00
(CG, it. 0), #cycles with negative reduced cost: 0


And we compare it with the approach based on full enumeration:

In [28]:
cycles_cf = er.find_all_cycles(aplus, max_length=4, cap=None)
sol, stime, duals = er.cycle_formulation(pairs, cycles_cf, tlim=10, verbose=0, relaxation=True)
print(f'(Full formulation) #cycles: {len(cycles_cf)}, time: {stime}, relaxation objective: {sol["objective"]:.2f}')

(Full formulation) #cycles: 9890, time: 1.29, relaxation objective: -36.00


## Column Generation - Downstream IP

**After we solve the CG formulation, we still don't have an actual solution**

* We have an optimal solution _of the LP relaxation_
* ...Which may violate the integrality constraints

**A simple strategy: keep the set of variables and solve the original problem**

In [29]:
sol, tme, _ = er.cycle_formulation(pairs, cycles_cg, tlim=30, verbose=1)

Solution time: 0.048, objective value: 36.0 (optimal)


This one is guarantee optimal _only if the LP-IP gap is zero_ (as in our case)

* Otherwise, we should start branching (leading to Branch & Price approaches)
* In practice, if the master problem formulation has a good LP bound
* ...Then even this simple sequential approach will lead good results

## Column Generation - Scalability

**Now we will quickly test the method scalability**

Let's try with 300 and 600 pairs:

In [30]:
%%time
pairs2, arcs2, aplus2 = er.generate_compatibility_graph(size=300, seed=2)
cycles_cg2, _  = er.cycle_formulation_cg(pairs2, aplus2, max_len=4, itcap=10)
_, _, _ = er.cycle_formulation(pairs2, cycles_cg2, tlim=30, verbose=1)

(CG, it. 0), #cycles: 8906, time: 0.547, relaxation objective: -122.00
(CG, it. 0), #cycles with negative reduced cost: 0
Solution time: 0.721, objective value: 122.0 (optimal)
CPU times: user 2.72 s, sys: 6.82 ms, total: 2.73 s
Wall time: 2.73 s


In [31]:
%%time
pairs3, arcs3, aplus3 = er.generate_compatibility_graph(size=600, seed=2)
cycles_cg3, _  = er.cycle_formulation_cg(pairs3, aplus3, max_len=4, itcap=10)
_, _, _ = er.cycle_formulation(pairs3, cycles_cg3, tlim=30, verbose=1)

(CG, it. 0), #cycles: 31010, time: 1.891, relaxation objective: -229.00
(CG, it. 0), #cycles with negative reduced cost: 0
Solution time: 2.349, objective value: 229.0 (optimal)
CPU times: user 13.4 s, sys: 85.6 ms, total: 13.4 s
Wall time: 13.5 s


## Considerations

**Column Generation can be a very useful technique**

* It is little known outside the Combinatorial Optimization community
* ...But it can provide massive scalability improvements
* ...And it can lead to simpler problem formulations

**The key is a clean master problem**

* The method works well if the master problem has a high-quality LP relaxation
* Usually, this is the case if the master has a clean structure
  - E.g. multi-knapsack, set covering, assignment problems...
* The trick is packing the complexity _in the definition of the problem variables_

**Combinatorial optimization can be used also in the pricing problem**

* If pricing problem does not admit clean formulation...
* ...You can still try and solve it using a combinatorial method (e.g. CP, SMT, MIP...) 